In [1]:
import numpy as np
import cv2

class conv_layer():

    # Sebagai inisialisasi layer convolusi
    def __init__(self, total_filters, filter_shape, lr, padding = 'no_padding', stride = 1, beta1 = 0.9, beta2 = 0.999):
        self.filters_ws = np.random.randn(*filter_shape, total_filters) * 0.1
        self.filter_bs = np.random.randn(total_filters) * 0.1
        self.padding = padding
        self.stride = stride
        self.din_dw = None 
        self.din_db = None
        self.input = None
        self.mo = 0
        self.acc = 0
        self.mo_b = 0
        self.acc_b = 0
        self.beta1 = beta1
        self.beta2 = beta2
        self.lr = lr

    def forward(self, input, training):
        #print(self.filters_ws)
        self.input = np.array(input, copy=True) # Input disimpan untuk digunakan nanti pada backward pass.


        ############### getting output dimentions here ###############
        
        # Dapatkan Dimensi Input dan Filter:
        n, input_dim_h, input_dim_w, _ = input.shape
        filter_dim_h, filter_dim_w, _, filter_dim_n = self.filters_ws.shape
        
        # Hitung Dimensi Output dan Padding:
        # Jika padding pd inputan convolusi
        # tujuan utamanya adalah mempertahankan dimensi tinggi dan lebar gambar input tetap sama setelah operasi konvolusi.
        if self.padding == 'keep_img_dim':
            output_shape = n, input_dim_h, input_dim_w, filter_dim_n
            filter_dim_h, filter_dim_w, _, _ = self.filters_ws.shape
            p_value = (filter_dim_h - 1) // 2, (filter_dim_w - 1) // 2
        # Jika padding pada inputan convolusi 'no_padding', dimensi output dihitung berdasarkan stride dan ukuran filter:
        elif self.padding == 'no_padding':
            out_dim_h = (input_dim_h - filter_dim_h) // self.stride + 1
            out_dim_w = (input_dim_w - filter_dim_w) // self.stride + 1
            output_shape = n, out_dim_h, out_dim_w, filter_dim_n
            p_value = 0, 0 # (0 tinggi, 0 lebar)
        ############### got output dimentions ###############

        out_dim_n, out_dim_h, out_dim_w, out_dim_c = output_shape

        input_padded = self.pad(input, p_value)
        output = np.zeros(output_shape)

        # Proses pergerakan kernel/filter
        for i in range(out_dim_h): # vertikal (tinggi)
            for j in range(out_dim_w): # horizontal (lebar)
                start_pix_x = i * self.stride # sumbu-x (vertikal)
                end_pix_x = start_pix_x + filter_dim_h
                start_pix_y = j * self.stride # sumbu-y (horizontal)
                end_pix_y = start_pix_y + filter_dim_w

                output[:, i, j, :] = np.sum(
                    input_padded[:, start_pix_x:end_pix_x, start_pix_y:end_pix_y, :, np.newaxis] *
                    self.filters_ws[np.newaxis, :, :, :],
                    axis=(1, 2, 3)
                )

        #print(output)
        return output + self.filter_bs

    @staticmethod
    def pad(array, pad):
        return np.pad(
            array=array, # Array input yang akan di-pad.
            pad_width=((0, 0), (pad[0], pad[0]), (pad[1], pad[1]), (0, 0)),
            mode='constant'
        )

In [2]:
class pool_layer():

    def __init__(self, input_dim, stride = 2):
        self.pool_dim = input_dim # Menyimpan dimensi pooling sebagai atribut kelas.
        self.stride = stride # Menyimpan nilai stride sebagai atribut kelas.
        self.input = None # Inisialisasi variabel untuk menyimpan input selama forward pass.
        self.max_pixels = {} # Inisialisasi dictionary untuk menyimpan lokasi nilai maksimum selama forward pass untuk digunakan dalam backward pass.

    def forward(self, input, training):
        #print(input.shape)
        self.input = np.array(input, copy=True) # Menyimpan salinan dari input untuk digunakan dalam backward pass.
        n, input_dim_h, input_dim_w, c = input.shape # Mendapatkan bentuk dari input (batch size, height, width, channels).
        pool_x_dim, pool_y_dim = self.pool_dim # Mendapatkan dimensi pooling (tinggi dan lebar).
        out_dim_h = 1 + (input_dim_h - pool_x_dim) // self.stride # Menghitung dimensi tinggi output setelah pooling.
        out_dim_w = 1 + (input_dim_w - pool_y_dim) // self.stride # Menghitung dimensi lebar output setelah pooling.
        output = np.zeros((n, out_dim_h, out_dim_w, c)) # Menginisialisasi output dengan nol.

        for i in range(out_dim_h):
            for j in range(out_dim_w):
                start_pix_x = i * self.stride # Menentukan koordinat awal pada sumbu x untuk area pooling.
                end_pix_x = start_pix_x + pool_x_dim # Menentukan koordinat akhir pada sumbu x untuk area pooling.
                start_pix_y = j * self.stride # Menentukan koordinat awal pada sumbu y untuk area pooling.
                end_pix_y = start_pix_y + pool_y_dim # Menentukan koordinat akhir pada sumbu y untuk area pooling.
                focus_area = input[:, start_pix_x:end_pix_x, start_pix_y:end_pix_y, :] # Mengambil area fokus untuk pooling.
                self.store_max_pixels(focus_area, (i, j)) # Menyimpan lokasi nilai maksimum dari area fokus.
                output[:, i, j, :] = np.max(focus_area, axis=(1, 2)) # mengekstrak nilai maksimum dari setiap area pooling

        #print(output)
        return output

    def store_max_pixels(self, area_pixels, i_j_location):
        mark_max = np.zeros_like(area_pixels) # Menginisialisasi array nol dengan bentuk yang sama seperti area pixels.
        n, h, w, c = area_pixels.shape # Mendapatkan bentuk dari area pixels.
        area_pixels = area_pixels.reshape(n, h * w, c) # Mereset area pixels ke dalam bentuk yang mudah untuk menemukan nilai maksimum.
        max_locations = np.argmax(area_pixels, axis=1) # Menemukan lokasi nilai maksimum dalam area pooling.
        n_idx, c_idx = np.indices((n, c)) # Membuat indeks untuk batch dan kanal.
        mark_max.reshape(n, h * w, c)[n_idx, max_locations, c_idx] = 1 # Menandai lokasi nilai maksimum.
        self.max_pixels[i_j_location] = mark_max # Menyimpan tanda lokasi maksimum dalam dictionary.

In [3]:
class reshape_layer():

    def __init__(self):
        self.input_shape = () # Inisialisasi variabel instance input_shape sebagai tuple kosong. Variabel ini akan digunakan untuk menyimpan bentuk (shape) dari input yang diterima oleh layer ini.

    def forward(self, input, training):
        self.input_shape = input.shape # Menyimpan bentuk (shape) dari input ke dalam self.input_shape.
        return np.ravel(input).reshape(input.shape[0], -1)
        # Menggunakan np.ravel(input) untuk mengubah input menjadi array 1D (flattening).
        # Menggunakan .reshape(input.shape[0], -1) untuk mengubah array 1D tersebut kembali menjadi 2D dengan jumlah baris sama dengan jumlah input awal (input.shape[0]) dan jumlah kolom sebanyak yang diperlukan untuk mempertahankan jumlah elemen total.

In [4]:
# all_ws = []
class weights_layer():
    def __init__(self, fan_in, fan_out, lr, beta1 = 0.9, beta2 = 0.999 , lamdaa = 0.0001):
        self.lamdaa = lamdaa
        self.lr = lr
        self.ws = np.random.randn(fan_in, fan_out)/np.sqrt(fan_in)
        self.bs = np.zeros(fan_out) #  Inisialisasi bias dengan nol
        
        # Inisialisasi momen dan akumulator untuk optimasi Adam.
        self.mo = 0
        self.acc = 0
        self.mo_b = 0
        self.acc_b = 0
        
        # Inisialisasi parameter beta1 dan beta2 untuk optimasi Adam.
        self.beta1 = beta1
        self.beta2 = beta2
        
    def forward(self,input, training):
        #all_ws.append(self.ws)
        #print(input.shape)
        #print(np.dot(input,self.ws) + self.bs)
        return np.dot(input,self.ws) + self.bs # Rumus Forward Propagation/Fully Connected Layer, Menghitung output dengan mengalikan input dengan bobot (self.ws) dan menambahkan bias (self.bs).

In [5]:
class ReLU():
    def __init__(self):
        pass
    
    def forward(self, input, training):
        relu_forward = np.maximum(0,input) # setiap elemen input yang kurang dari 0 diubah menjadi 0.
        return relu_forward


In [6]:
# Negative Log-Likelihood
def NLL(expected_probabilities,actual_labels):

    # Menentukan Probabilitas Benar
    correct_prob = expected_probabilities[np.arange(len(expected_probabilities)),actual_labels]

    p = np.exp(correct_prob) / np.sum(np.exp(expected_probabilities),axis=-1) # Implementasi softmax

    loss = -1 * np.log(p) # NLL
  
    return loss

In [7]:
def run_batch(CNN, X, training):
    all_layers_outputs = []  # Membuat list kosong untuk menyimpan output dari setiap layer
    received = X  # Menerima input awal yaitu X (input batch)
    for layer in CNN:
        all_layers_outputs.append(layer.forward(received, training))  # Menjalankan forward pass untuk setiap layer
        received = all_layers_outputs[-1]  # Menyimpan output terakhir sebagai input untuk layer berikutnya
  
    assert len(all_layers_outputs) == len(CNN)  # Memastikan jumlah output yang dihasilkan sesuai dengan jumlah layer
    
    return all_layers_outputs  # Mengembalikan semua output dari setiap layer

In [8]:
class dropout_layer():

    def __init__(self, keep_prob):
        self.cutoff_prob = keep_prob  # Probabilitas untuk mempertahankan neuron
        self.zeros_for_dropped = None  # Mask untuk neuron yang dijatuhkan

    def forward(self, input, training):
        if training:  # Jika dalam mode training
            self.zeros_for_dropped = (np.random.rand(*input.shape) < self.cutoff_prob)  # Membuat mask dropout, dimana jika nilai input kurang dari nilai keep_prob pd dropout, neuron dijatuhkan
            return self.drop(input, self.zeros_for_dropped)  # Terapkan dropout pada input
        else:
            return input  # Jika tidak dalam mode training, kembalikan input tanpa perubahan

    def drop(self, input, zeros_for_dropped):
        input *= zeros_for_dropped  # Set nilai neuron yang dijatuhkan menjadi 0
        input /= self.cutoff_prob  # Skala ulang input untuk mempertahankan ekspektasi nilai
        return input  # Kembalikan input yang sudah diterapkan dropout

In [9]:
# import os
# import pickle
# import pygame
# from flask import Flask, render_template, request, redirect, url_for, flash
# from werkzeug.utils import secure_filename
# import logging

# # Initialize pygame mixer for sound
# pygame.mixer.init()

# # Create a Flask application instance
# app = Flask(__name__, static_folder='static')
# app.config['UPLOAD_FOLDER'] = 'static/uploads'
# app.secret_key = 'secret_key'

# # Initialize logging
# logging.basicConfig(level=logging.INFO)

# current_image_name = None
# model_notification = None

# # Hardcode: Load the model from a pickle file at startup
# MODEL_PATH = 'static/models/trained_model_manual_10000002_32_7_resize64_arsitekturbaru.pkl'

# def load_model():
#     with open(MODEL_PATH, 'rb') as file:
#         model = pickle.load(file)
#     return model

# # Load the model at startup
# try:
#     CNN_loaded = load_model()
#     model_notification = "Model berhasil dimuat dari sistem."
# except Exception as e:
#     CNN_loaded = None
#     model_notification = f"Error memuat model: {str(e)}"
#     logging.error(model_notification)

# # Function to delete all files in the upload folder
# def delete_all_files_in_upload_folder():
#     files = os.listdir(app.config['UPLOAD_FOLDER'])
#     if not files:
#         logging.info("Folder uploads is empty or does not exist.")
#     for file in files:
#         file_path = os.path.join(app.config['UPLOAD_FOLDER'], file)
#         if os.path.isfile(file_path):
#             try:
#                 os.remove(file_path)
#                 logging.info(f"Deleted file: {file_path}")
#             except Exception as e:
#                 logging.error(f"Error deleting file {file_path}: {e}")

# # Fungsi untuk memprediksi gambar menggunakan model CNN
# def predict_image(model, image_path):
#     img = cv2.imread(image_path)
#     if img is None:
#         raise ValueError("Gambar tidak dapat dibaca.")
    
#     # Sesuaikan dengan ukuran input model (32x32 di sini)
#     img = cv2.resize(img, (64, 64))
#     img = img.astype(float) / 255.0
#     img = np.expand_dims(img, axis=0)  # Tambahkan dimensi batch
    
#     # Melakukan prediksi menggunakan model
#     # Pastikan model Anda memiliki fungsi prediksi yang sesuai dengan format ini
#     predictions = run_batch(model, img, training=False)[-1]
    
#     # Terapkan softmax pada output
#     softmax_probabilities = np.exp(predictions) / np.sum(np.exp(predictions))
    
#     # Tentukan ambang batas
#     threshold = 0.7
#     prediction = softmax_probabilities.argmax(axis=-1)[0]
#     confidence = softmax_probabilities.max()

#     # Menentukan label prediksi
#     labels = ['blight', 'blast', 'tungro', 'healthy']
    
#     # Menghitung akurasi sebagai nilai kepercayaan (confidence)
#     accuracy = confidence * 100  # Konversi ke persentase
    
#     # **Perbaikan: pastikan probabilities 1D**
#     softmax_probabilities = softmax_probabilities.flatten().tolist()

#     # Debug: cek output setelah flatten
#     print("Softmax probabilities (flattened):", softmax_probabilities)
    
#     # Mengembalikan prediksi dan akurasi
#     if confidence < threshold:
#         return 'bukan daun padi', accuracy, softmax_probabilities
#     else:
#         return labels[prediction], accuracy, softmax_probabilities

# # Play sound based on prediction
# def play_sound(prediction):
#     sounds = {
#         "tungro": "sounds/tungro.wav",
#         "blast": "sounds/blast.wav",
#         "blight": "sounds/blight.wav",
#         "healthy": "sounds/healthy.wav",
#         "bukan daun padi": "sounds/not.mp3"
#     }
#     sound_file = os.path.join('static', sounds.get(prediction.lower(), "healthy.wav"))
#     if sound_file and os.path.exists(sound_file):
#         pygame.mixer.music.load(sound_file)
#         pygame.mixer.music.play()

# # Define routes
# @app.route('/')
# def index():
#     delete_all_files_in_upload_folder()
#     global current_image_name, model_notification
#     return render_template('index.html', 
#                            image_name=current_image_name,
#                            model_notification=model_notification)

# @app.route('/predict', methods=['POST'])
# def predict():
#     global CNN_loaded, current_image_name
#     if CNN_loaded is None:
#         flash('Model belum dimuat. Harap muat model terlebih dahulu.')
#         return redirect(url_for('index'))

#     if 'image_file' not in request.files:
#         flash('Tidak ada file gambar yang dipilih.')
#         return redirect(url_for('index'))

#     file = request.files['image_file']
#     if file.filename == '':
#         flash('Tidak ada file gambar yang dipilih.')
#         return redirect(url_for('index'))

#     try:
#         # Log file name
#         app.logger.info(f"Received file: {file.filename}")

#         # Save the uploaded image
#         filename = secure_filename(file.filename)  # Pastikan secure_filename diimpor
#         app.logger.info(f"Secure filename: {filename}")
        
#         file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
#         file.save(file_path)
#         current_image_name = filename

#         # Perform prediction
#         prediction, accuracy, probabilities = predict_image(CNN_loaded, file_path)
#         play_sound(prediction)

#         # Return prediction results
#         return render_template('index.html', 
#                                prediction=prediction, 
#                                accuracy=accuracy,  # Pass accuracy to the template
#                                probabilities=probabilities,  # Tambahkan ini
#                                labels=['blight', 'blast', 'tungro', 'healthy'],  # Label untuk menampilkan hasil
#                                image_filename=filename,
#                                image_name=current_image_name)
#     except Exception as e:
#         app.logger.error(f"Error in prediction: {str(e)}")
#         flash(f"Error: {str(e)}")
#         return redirect(url_for('index'))

# # Run the Flask app
# if __name__ == '__main__':
#     app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

**JSON**

In [10]:
# import os
# import pickle
# import pygame
# from flask import Flask, request, jsonify, redirect, url_for, flash, render_template
# from flask_cors import CORS
# from werkzeug.utils import secure_filename
# import logging
# import cv2
# import numpy as np
# import datetime
# from collections import defaultdict
# import uuid
# from calendar import month_abbr

# # =========================================================================
# # Definisi kelas dan fungsi kustom model Anda ada di file yang sama
# # Jadi tidak perlu didefinisikan ulang di sini.
# # Pastikan Anda menjalankan seluruh cell notebook sebelum memanggil API.
# # =========================================================================

# # Inisialisasi pygame mixer untuk suara
# pygame.mixer.init()

# # Buat instance aplikasi Flask
# app = Flask(__name__, static_folder='static')
# # Konfigurasi CORS yang lebih spesifik untuk mengizinkan semua
# CORS(app, resources={r"/*": {"origins": "*"}})
# app.config['UPLOAD_FOLDER'] = 'static/uploads'
# app.secret_key = 'secret_key'

# # Inisialisasi logging
# logging.basicConfig(level=logging.INFO)

# current_image_name = None
# model_notification = None

# # Hardcode: Muat model dari file pickle saat startup
# MODEL_PATH = 'static/models/trained_model_manual_10000002_32_7_resize64_arsitekturbaru.pkl'

# # --- DEMO DATABASE (Data di Memori) ---
# # Ini akan hilang setiap kali server di-restart. Gunakan database sungguhan
# # untuk data yang persisten (misal: Firestore, SQLite).
# dashboard_data = []

# def load_model():
#     """Memuat model dari file pickle."""
#     try:
#         with open(MODEL_PATH, 'rb') as file:
#             model = pickle.load(file)
#         return model
#     except FileNotFoundError:
#         logging.error(f"File model tidak ditemukan di: {MODEL_PATH}")
#         return None
#     except Exception as e:
#         logging.error(f"Error memuat model: {str(e)}")
#         return None

# # Muat model saat startup
# CNN_loaded = load_model()
# if CNN_loaded:
#     model_notification = "Model berhasil dimuat dari sistem."
# else:
#     model_notification = "Error: Model tidak dapat dimuat."

# # Fungsi untuk menghapus semua file di folder unggahan
# def delete_all_files_in_upload_folder():
#     """Menghapus semua file di folder unggahan."""
#     files = os.listdir(app.config['UPLOAD_FOLDER'])
#     for file in files:
#         file_path = os.path.join(app.config['UPLOAD_FOLDER'], file)
#         if os.path.isfile(file_path):
#             try:
#                 os.remove(file_path)
#             except Exception as e:
#                 logging.error(f"Error menghapus file {file_path}: {e}")

# def predict_image(model, image_path):
#     """
#     Memuat gambar, melakukan preprocessing, dan memprediksinya
#     menggunakan model yang dimuat.
#     """
#     img = cv2.imread(image_path)
#     if img is None:
#         raise ValueError("Gambar tidak dapat dibaca.")
    
#     img = cv2.resize(img, (64, 64))
#     img = img.astype(float) / 255.0
#     img = np.expand_dims(img, axis=0)
    
#     try:
#         predictions = run_batch(model, img, training=False)[-1]
#     except Exception as e:
#         logging.error(f"Error calling run_batch: {e}")
#         raise Exception(f"Gagal memprediksi dengan model: {e}")

#     softmax_probabilities = np.exp(predictions) / np.sum(np.exp(predictions))
    
#     threshold = 0.7
#     prediction_index = np.argmax(softmax_probabilities)
#     confidence = np.max(softmax_probabilities)
    
#     labels = ['blight', 'blast', 'tungro', 'healthy']
    
#     softmax_probabilities = softmax_probabilities.flatten().tolist()
    
#     if confidence < threshold:
#         return 'bukan daun padi', confidence, softmax_probabilities, labels
#     else:
#         return labels[prediction_index], confidence, softmax_probabilities, labels

# def play_sound(prediction):
#     """Memainkan suara berdasarkan hasil prediksi."""
#     sounds = {
#         "tungro": "sounds/tungro.wav",
#         "blast": "sounds/blast.wav",
#         "blight": "sounds/blight.wav",
#         "healthy": "sounds/healthy.wav",
#         "bukan daun padi": "sounds/not.mp3"
#     }
#     sound_file = os.path.join('static', sounds.get(prediction.lower(), "healthy.wav"))
#     if os.path.exists(sound_file):
#         pygame.mixer.music.load(sound_file)
#         pygame.mixer.music.play()

# @app.route('/predict_api', methods=['POST'])
# def predict_api():
#     if CNN_loaded is None:
#         return jsonify({'error': 'Model belum dimuat. Harap muat model terlebih dahulu.'}), 500

#     if 'image' not in request.files:
#         return jsonify({'error': 'Tidak ada file gambar yang dipilih.'}), 400

#     file = request.files['image']
#     if file.filename == '':
#         return jsonify({'error': 'Tidak ada file gambar yang dipilih.'}), 400

#     try:
#         # Buat nama file yang unik untuk menghindari tabrakan
#         unique_filename = f"{uuid.uuid4()}_{secure_filename(file.filename)}"
#         file_path = os.path.join(app.config['UPLOAD_FOLDER'], unique_filename)
#         file.save(file_path)
        
#         prediction_label, confidence, probabilities, labels = predict_image(CNN_loaded, file_path)
        
#         # Tentukan tingkat keparahan berdasarkan label
#         if prediction_label == 'healthy':
#             severity = 'low'
#         elif confidence > 0.95:
#             severity = 'high'
#         else:
#             severity = 'medium'
        
#         new_result = {
#             'id': str(uuid.uuid4()), # ID unik untuk setiap riwayat
#             'timestamp': datetime.datetime.now().isoformat(),
#             'prediction': prediction_label,
#             'confidence': confidence,
#             'probabilities': probabilities,
#             'labels': labels,
#             'image_filename': unique_filename,
#             'severity': severity,
#         }
#         dashboard_data.append(new_result)
        
#         return jsonify(new_result)

#     except Exception as e:
#         logging.error(f"Error in prediction: {str(e)}")
#         return jsonify({'error': str(e)}), 500

# # -----------------
# # API BARU UNTUK RIWAYAT KLASIFIKASI
# # -----------------
# @app.route('/api/history', methods=['GET'])
# def get_history_data():
#     history_list = []
#     upload_folder = app.config['UPLOAD_FOLDER']
    
#     # Pesan ini akan muncul sekali setiap kali /api/history dipanggil
#     logging.info("Mulai memproses permintaan riwayat...") 

#     for item in dashboard_data:
#         full_image_path = os.path.join(upload_folder, item['image_filename'])
        
#         # ==========================================================
#         # TAMBAHKAN BARIS INI UNTUK MENAMPILKAN PATH DI TERMINAL
#         logging.info(f"Mengecek file di path: {full_image_path}")
#         # ==========================================================

#         if os.path.exists(full_image_path):
#             logging.info(f"   -> DITEMUKAN: {item['image_filename']}") # Tambahan: konfirmasi jika file ada
            
#             image_url_path = f"/static/uploads/{item['image_filename']}"
            
#             history_list.append({
#                 'id': item['id'],
#                 'date': datetime.datetime.fromisoformat(item['timestamp']).strftime('%Y-%m-%d %H:%M:%S'),
#                 'image': image_url_path,
#                 'disease': item['prediction'],
#                 'confidence': round(item['confidence'] * 100, 2),
#                 'severity': item['severity'],
#                 'image_filename': item['image_filename']
#             })
#         else:
#             logging.warning(f"   -> TIDAK DITEMUKAN: {item['image_filename']}") # Tambahan: konfirmasi jika file tidak ada

#     history_list.sort(key=lambda x: x['date'], reverse=True)
#     return jsonify(history_list)

# # -----------------
# # API BARU UNTUK DASHBOARD
# # -----------------
# @app.route('/api/dashboard', methods=['GET'])
# def get_dashboard_data():
#     """
#     Mengambil data prediksi dari 'database' dan memprosesnya untuk dashboard.
#     """
#     # Buat daftar inisial untuk 12 bulan
#     monthly_counts = [{'month': abbr, 'total': 0, 'healthy': 0, 'diseases': 0} for abbr in month_abbr[1:]]
    
#     # Buat kamus untuk memudahkan pembaruan
#     monthly_data_map = {item['month']: item for item in monthly_counts}

#     if not dashboard_data:
#         return jsonify({
#             'total_classifications': 0,
#             'classifications_by_month': monthly_counts,
#             'classification_distribution': []
#         })

#     total_classifications = len(dashboard_data)

#     # Isi data dari dashboard_data
#     for item in dashboard_data:
#         month = datetime.datetime.fromisoformat(item['timestamp']).strftime('%b')
#         if month in monthly_data_map:
#             monthly_data_map[month]['total'] += 1
#             if item['prediction'] == 'healthy':
#                 monthly_data_map[month]['healthy'] += 1
#             else:
#                 monthly_data_map[month]['diseases'] += 1
    
#     # Ubah kembali ke format daftar
#     updated_monthly_counts = list(monthly_data_map.values())
    
#     classification_counts = defaultdict(int)
#     for item in dashboard_data:
#         classification_counts[item['prediction']] += 1
    
#     distribution = [{'label': k, 'count': v} for k, v in classification_counts.items()]

#     return jsonify({
#         'total_classifications': total_classifications,
#         'classifications_by_month': updated_monthly_counts,
#         'classification_distribution': distribution
#     })
    
# # Tambahkan ini di bagian ENDPOINTS API di file app.ipynb Anda

# @app.route('/')
# def home():
#     """Endpoint dasar untuk mengecek status API."""
#     return jsonify({'status': 'ok', 'message': 'Rice Doctor AI API is running!'})


# if __name__ == '__main__':
#     app.run(host='0.0.0.0', port=5000, debug=False)

**SQLite3**

In [ ]:
import os
import pickle
import sqlite3
from flask import Flask, request, jsonify
from flask_cors import CORS
from werkzeug.utils import secure_filename
import logging
import cv2
import numpy as np
import datetime
import uuid
from calendar import month_abbr

# =========================================================================
# Pastikan definisi kelas dan fungsi kustom model Anda (seperti run_batch)
# sudah dijalankan di cell notebook sebelum menjalankan cell ini.
# =========================================================================

# --- Konfigurasi Aplikasi & Database ---
app = Flask(__name__, static_folder='static')
CORS(app, resources={r"/*": {"origins": "*"}})
app.config['UPLOAD_FOLDER'] = 'static/uploads'
DATABASE = 'history.db' # Nama file database SQLite
logging.basicConfig(level=logging.INFO)

def get_db_connection():
    """Membuat koneksi ke database SQLite."""
    conn = sqlite3.connect(DATABASE)
    conn.row_factory = sqlite3.Row # Memungkinkan akses kolom berdasarkan nama
    return conn

def init_db():
    """Membuat tabel database jika belum ada."""
    with app.app_context():
        conn = get_db_connection()
        with open('schema.sql', 'r') as f:
            conn.executescript(f.read())
        conn.close()
        logging.info("Database berhasil diinisialisasi.")

# --- Memuat Model Machine Learning ---
MODEL_PATH = 'static/models/trained_model_manual_10000002_32_7_resize64_arsitekturbaru.pkl'
CNN_loaded = None

def load_model():
    """Memuat model dari file pickle."""
    try:
        with open(MODEL_PATH, 'rb') as file:
            model = pickle.load(file)
            logging.info(f"Model berhasil dimuat dari: {MODEL_PATH}")
            return model
    except Exception as e:
        logging.error(f"Error saat memuat model: {e}")
        return None

CNN_loaded = load_model()

# --- Fungsi Helper untuk Prediksi ---
def predict_image(model, image_path):
    """
    Memuat gambar, melakukan preprocessing, dan memprediksinya
    menggunakan model yang dimuat.
    """
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError("Gambar tidak dapat dibaca.")
    
    img = cv2.resize(img, (64, 64))
    img = img.astype(float) / 255.0
    img = np.expand_dims(img, axis=0)
    
    try:
        predictions = run_batch(model, img, training=False)[-1]
    except Exception as e:
        logging.error(f"Error calling run_batch: {e}")
        raise Exception(f"Gagal memprediksi dengan model: {e}")

    softmax_probabilities = np.exp(predictions) / np.sum(np.exp(predictions))
    
    threshold = 0.7
    prediction_index = np.argmax(softmax_probabilities)
    confidence = np.max(softmax_probabilities)
    
    labels = ['blight', 'blast', 'tungro', 'healthy']
    
    softmax_probabilities = softmax_probabilities.flatten().tolist()
    
    if confidence < threshold:
        return 'bukan daun padi', confidence, softmax_probabilities, labels
    else:
        return labels[prediction_index], confidence, softmax_probabilities, labels

# --- ENDPOINTS API ---

@app.route('/')
def home():
    """Endpoint dasar untuk mengecek status API."""
    return jsonify({'status': 'ok', 'message': 'Rice Doctor AI API is running!'})

@app.route('/predict_api', methods=['POST'])
def predict_api():
    """Endpoint untuk menerima gambar, memprediksi, dan menyimpan ke DB."""
    if CNN_loaded is None:
        return jsonify({'error': 'Model tidak tersedia di server.'}), 500
    if 'image' not in request.files:
        return jsonify({'error': 'Tidak ada file gambar yang dikirim.'}), 400

    file = request.files['image']
    if file.filename == '':
        return jsonify({'error': 'File gambar tidak valid.'}), 400

    try:
        unique_filename = f"{uuid.uuid4()}_{secure_filename(file.filename)}"
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], unique_filename)
        file.save(file_path)
        
        prediction_label, confidence, probabilities, labels = predict_image(CNN_loaded, file_path)
        
        if prediction_label in ['healthy', 'bukan daun padi']:
            severity = 'low'
        elif confidence > 0.95:
            severity = 'high'
        else:
            severity = 'medium'
            
        # Simpan hasil ke database SQLite
        conn = get_db_connection()
        item_id = str(uuid.uuid4())
        timestamp = datetime.datetime.now().isoformat()
        
        conn.execute(
            'INSERT INTO history (id, timestamp, prediction, confidence, image_filename, severity) VALUES (?, ?, ?, ?, ?, ?)',
            (item_id, timestamp, prediction_label, confidence, unique_filename, severity)
        )
        conn.commit()
        conn.close()
        
        # Kirim kembali hasil yang baru saja disimpan
        return jsonify({
            'id': item_id,
            'date': datetime.datetime.fromisoformat(timestamp).strftime('%Y-%m-%d %H:%M:%S'),
            'image': f"/static/uploads/{unique_filename}",
            'prediction': prediction_label, # Ganti 'disease' menjadi 'prediction' agar konsisten
            'confidence': confidence,
            'severity': severity,
            'image_filename': unique_filename,
            'probabilities': probabilities, 
            'labels': labels              
        })

    except Exception as e:
        logging.error(f"Error saat prediksi: {e}")
        return jsonify({'error': f'Terjadi kesalahan di server: {e}'}), 500

@app.route('/api/history', methods=['GET'])
def get_history():
    """Endpoint untuk mendapatkan semua riwayat dari database."""
    conn = get_db_connection()
    # Mengambil semua data dari tabel history, diurutkan dari yang terbaru
    history_rows = conn.execute('SELECT * FROM history ORDER BY timestamp DESC').fetchall()
    conn.close()
    
    history_list = [dict(row) for row in history_rows]
    
    # Format data agar sesuai dengan yang diharapkan frontend
    formatted_list = [
        {
            'id': item['id'],
            'date': datetime.datetime.fromisoformat(item['timestamp']).strftime('%Y-%m-%d %H:%M:%S'),
            'image': f"/static/uploads/{item['image_filename']}",
            'disease': item['prediction'],
            'confidence': item['confidence'] * 100,
            'severity': item['severity'],
            'image_filename': item['image_filename']
        }
        for item in history_list
    ]
    return jsonify(formatted_list)

@app.route('/api/dashboard', methods=['GET'])
def get_dashboard_data():
    """Endpoint untuk data agregat dashboard dari database."""
    conn = get_db_connection()
    # Ambil 'prediction' dan 'timestamp' dari database
    rows = conn.execute('SELECT prediction, timestamp FROM history').fetchall()
    conn.close()
    
    # Buat daftar inisial untuk 12 bulan
    # month_abbr[1:] akan menghasilkan ['Jan', 'Feb', ..., 'Dec']
    monthly_counts = [{'month': abbr, 'total': 0, 'healthy': 0, 'diseases': 0} for abbr in month_abbr[1:]]
    monthly_data_map = {item['month']: item for item in monthly_counts}

    if not rows:
        return jsonify({
            'total_classifications': 0,
            'classifications_by_month': monthly_counts, # Kirim data bulan kosong
            'classification_distribution': []
        })

    # Hitung total dan distribusi
    total_classifications = len(rows)
    distribution_counts = {}
    
    # Proses setiap baris data
    for row in rows:
        prediction = row['prediction']
        
        # Hitung distribusi
        distribution_counts[prediction] = distribution_counts.get(prediction, 0) + 1
        
        # Hitung data bulanan
        month_str = datetime.datetime.fromisoformat(row['timestamp']).strftime('%b') # -> 'Jan', 'Feb', dst.
        if month_str in monthly_data_map:
            monthly_data_map[month_str]['total'] += 1
            if prediction == 'healthy':
                monthly_data_map[month_str]['healthy'] += 1
            else:
                monthly_data_map[month_str]['diseases'] += 1

    distribution = [{'label': k, 'count': v} for k, v in distribution_counts.items()]
    
    # Kirim kembali data lengkap termasuk data bulanan
    return jsonify({
        'total_classifications': total_classifications,
        'classifications_by_month': list(monthly_data_map.values()), # <-- DATA BARU
        'classification_distribution': distribution
    })


if __name__ == '__main__':
    # Pastikan folder uploads sudah ada
    if not os.path.exists(app.config['UPLOAD_FOLDER']):
        os.makedirs(app.config['UPLOAD_FOLDER'])
    
    # Inisialisasi database sebelum menjalankan aplikasi
    init_db()
    
    app.run(host='0.0.0.0', port=5000, debug=False)

INFO:root:Model berhasil dimuat dari: static/models/trained_model_manual_10000002_32_7_resize64_arsitekturbaru.pkl
INFO:root:Database berhasil diinisialisasi.


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.13:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [29/Aug/2025 09:54:38] "GET /api/history HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Aug/2025 09:54:38] "GET /api/history HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Aug/2025 09:54:38] "GET /api/dashboard HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Aug/2025 09:54:38] "GET /api/dashboard HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Aug/2025 09:54:44] "POST /predict_api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Aug/2025 09:54:45] "GET /api/dashboard HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Aug/2025 09:54:45] "GET /api/dashboard HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Aug/2025 09:54:46] "GET /api/history HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [2